In [ ]:
from google.colab import drive
import duckdb
import pandas as pd
import numpy as np

# Montar Drive
drive.mount('/content/drive')

# AJUSTA ESTA RUTA a donde descargues los archivos de IEEE-CIS
base_path_fraud = '/content/drive/My Drive/Colab Notebooks/Detección fraude/'

train_trans = base_path_fraud + 'train_transaction.csv'
train_id    = base_path_fraud + 'train_identity.csv'

# Conexión DuckDB
con = duckdb.connect(database=':memory:')

In [ ]:
# 1. Limpieza de Tiempo y Montos
# El tiempo viene en segundos. Sacaremos la hora y el día.
query_trans = f"""
CREATE OR REPLACE VIEW trans_clean AS
SELECT
    TransactionID,
    isFraud,
    TransactionAmt,
    TransactionDT,
    -- Ingeniería de Tiempo: Ciclos de 24h y 7 días
    CAST((TransactionDT / 3600) % 24 AS INT) AS hour_of_day,
    CAST((TransactionDT / 86400) % 7 AS INT) AS day_of_week,
    -- Variable de Negocio: ¿Es un monto redondo? (Frecuente en lavado/fraude)
    CASE WHEN TransactionAmt % 1 = 0 THEN 1 ELSE 0 END as is_round_amount,
    card1, card2, card3,
    P_emaildomain,
    dist1
FROM read_csv_auto('{train_trans}')
"""

# 2. Identidad: Dispositivo y Red
query_identity = f"""
CREATE OR REPLACE VIEW identity_clean AS
SELECT
    TransactionID,
    DeviceType,
    DeviceInfo,
    -- Agrupamos dispositivos por marcas comunes para reducir cardinalidad
    CASE
        WHEN DeviceInfo LIKE '%Windows%' THEN 'Windows'
        WHEN DeviceInfo LIKE '%iOS%' OR DeviceInfo LIKE '%iPhone%' THEN 'Apple'
        WHEN DeviceInfo LIKE '%Android%' OR DeviceInfo LIKE '%Samsung%' THEN 'Android'
        ELSE 'Other'
    END as device_brand
FROM read_csv_auto('{train_id}')
"""

# 3. Master Table: Unión (Aquí es donde ocurre la magia)
query_master_fraud = """
SELECT
    t.*,
    i.DeviceType,
    i.device_brand,
    -- Lógica de Email: El dominio puede indicar riesgo
    COALESCE(t.P_emaildomain, 'anonymous') as email
FROM trans_clean t
LEFT JOIN identity_clean i ON t.TransactionID = i.TransactionID
"""

# Ejecutar Pipeline de Fraude
con.execute(query_trans)
con.execute(query_identity)
df_fraud = con.execute(query_master_fraud).df()

print(f"✅ ¡Éxito! Dataset consolidado ({df_fraud.shape[0]},{df_fraud.shape[1]})")
# Diagnóstico del dataset
print(df_fraud.isnull().sum() / len(df_fraud)*100) # Porcentaje de nulos
display(df_fraud.head())

In [ ]:
def run_fraud_audit(con, tables):
    print(f"{' AUDITORÍA DE FRAUDE (IEEE-CIS) ':=^40}")
    for table in tables:
        res = con.execute(f"""
            SELECT
                '{table}' as tabla,
                COUNT(*) as total,
                SUM(CASE WHEN TransactionID IS NULL THEN 1 ELSE 0 END) as nulos_id,
                -- En fraude, nos interesa mucho el % de la clase positiva
                AVG(CASE WHEN isFraud = 1 THEN 1.0 ELSE 0.0 END) * 100 as tasa_fraude
            FROM {table}
        """).df()
        print(f"📊 {res['tabla'][0]} | Total: {res['total'][0]} | Fraude: {res['tasa_fraude'][0]:.2f}%")
    print("="*40)

# Ejecutar auditoría
# Nota: La tabla identity no tiene la columna isFraud, por eso solo auditamos la master
run_fraud_audit(con, ['df_fraud'])

In [ ]:
# Ingeniería de Variables: Velocity y Agregaciones
query_velocity = f"""
CREATE OR REPLACE VIEW features_velocity AS
SELECT
    TransactionID,
    -- 1. Frecuencia: ¿Cuántas transacciones ha hecho esta tarjeta (card1) en total?
    COUNT(*) OVER(PARTITION BY card1) as card1_cnt,

    -- 2. Desviación del Monto: ¿Qué tanto varía esta compra del promedio de la tarjeta?
    TransactionAmt / AVG(TransactionAmt) OVER(PARTITION BY card1) as amt_to_mean_card1,

    -- 3. Velocity Temporal: (Simulada con el orden de las transacciones)
    -- En un dataset real usaríamos intervalos de tiempo, aquí usamos conteos acumulados
    ROW_NUMBER() OVER(PARTITION BY card1 ORDER BY TransactionDT) as trans_count_total
FROM trans_clean
"""

# Unión Final de Features
query_final_model = """
SELECT
    m.*,
    v.card1_cnt,
    v.amt_to_mean_card1,
    v.trans_count_total
FROM df_fraud m -- Asegúrate que este nombre coincida con tu tabla anterior
LEFT JOIN features_velocity v ON m.TransactionID = v.TransactionID
"""

# Ejecutamos en DuckDB
con.execute(query_velocity)
df_final = con.execute(query_final_model).df()

print(f"✅ Dataset con Features de Velocity listo: {df_final.shape}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 5))

# Calculamos la proporción de fraude por hora
fraud_by_hour = df_fraud.groupby('hour_of_day')['isFraud'].mean()

sns.lineplot(x=fraud_by_hour.index, y=fraud_by_hour.values, marker='o', color='red', linewidth=2.5)
plt.fill_between(fraud_by_hour.index, fraud_by_hour.values, color='red', alpha=0.1)

plt.title('Probabilidad de Fraude según la Hora del Día', fontsize=14)
plt.xlabel('Hora (0-23)', fontsize=12)
plt.ylabel('% Tasa de Fraude', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(range(0, 24))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# Filtramos para ver la distribución de montos
sns.kdeplot(df_fraud[df_fraud['isFraud'] == 0]['TransactionAmt'], label='Legítima', fill=True, color='blue', log_scale=True)
sns.kdeplot(df_fraud[df_fraud['isFraud'] == 1]['TransactionAmt'], label='Fraude', fill=True, color='red', log_scale=True)

plt.title('Distribución de Montos: Legítimo vs Fraude (Escala Log)', fontsize=14)
plt.xlabel('Monto de Transacción (Log Scale)', fontsize=12)
plt.legend()
plt.show()

In [ ]:
# Creamos un subplot de 1x2
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# 1. Por Dispositivo
sns.barplot(data=df_fraud, x='device_brand', y='isFraud', ax=ax[0], palette='viridis', ci=None)
ax[0].set_title('Tasa de Fraude por Marca de Dispositivo')
ax[0].set_ylabel('% Fraude')

# 2. Por Dominio de Email
# Tomamos solo los top 5 para que sea legible
top_emails = df_fraud['email'].value_counts().nlargest(5).index
df_top_emails = df_fraud[df_fraud['email'].isin(top_emails)]

sns.barplot(data=df_top_emails, x='email', y='isFraud', ax=ax[1], palette='magma', ci=None)
ax[1].set_title('Tasa de Fraude por Proveedor de Email')
ax[1].set_ylabel('% Fraude')

plt.tight_layout()
plt.show()

In [ ]:
# 1. Ordenar el DataFrame por tiempo
df_final = df_final.sort_values('TransactionDT')

# 2. Calcular el punto de corte (80%)
split_idx = int(len(df_final) * 0.8)

# 3. Dividir
train_df = df_final.iloc[:split_idx]
test_df  = df_final.iloc[split_idx:]

print(f"📈 Entrenamiento: {train_df.shape[0]} transacciones (Pasado)")
print(f"📉 Prueba: {test_df.shape[0]} transacciones (Futuro)")

In [ ]:
# Definir variables (Features)
features = [
    'TransactionAmt', 'hour_of_day', 'day_of_week', 'is_round_amount',
    'card1', 'card2', 'dist1', 'card1_cnt', 'amt_to_mean_card1', 'trans_count_total'
]

# X e y para entrenamiento y prueba (usando el Time-Split anterior)
X_train = train_df[features]
y_train = train_df['isFraud']

X_test = test_df[features]
y_test = test_df['isFraud']

In [ ]:
class FraudConfig:
    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'eval_metric': 'aucpr',
        'use_label_encoder': False, # Recomendado en versiones recientes de XGBoost
        'n_estimators': 500,
        'learning_rate': 0.05,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'random_state': 42,
        'n_jobs': -1 # Usar todos los cores disponibles
    }

In [ ]:
import xgboost as xgb
from sklearn.metrics import average_precision_score, roc_auc_score

# --- MODELO A: XGBOOST ESTÁNDAR ---
model_a = xgb.XGBClassifier(**FraudConfig.XGB_PARAMS)
model_a.fit(X_train, y_train)

# --- MODELO B: XGBOOST BALANCEADO ---
# Calculamos el ratio de desbalanceo: (Negativos / Positivos)
ratio = (y_train == 0).sum() / (y_train == 1).sum()

model_b = xgb.XGBClassifier(
    **FraudConfig.XGB_PARAMS,
    scale_pos_weight=ratio # Penaliza más fuerte si falla en detectar un fraude
)
model_b.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt

# Usaremos el Modelo B por ser más sensible al riesgo
xgb.plot_importance(model_b, max_num_features=10, importance_type='gain',
                   title='Top 10 Variables Predictoras de Fraude')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

def plot_ks_gini(y_true, y_probs):
    # 1. Preparar datos
    df = pd.DataFrame({'target': y_true, 'proba': y_probs})
    df = df.sort_values(by='proba', ascending=False).reset_index(drop=True)

    # 2. Calcular acumulados
    df['event'] = df['target'].cumsum() / df['target'].sum()
    df['non_event'] = (1 - df['target']).cumsum() / (1 - df['target']).sum()
    df['ks_diff'] = abs(df['event'] - df['non_event'])

    # 3. Métricas
    ks_stat = df['ks_diff'].max()
    ks_idx = df['ks_diff'].idxmax()
    ks_threshold = df.loc[ks_idx, 'proba']

    auc = roc_auc_score(y_true, y_probs)
    gini = 2 * auc - 1

    # 4. Graficar
    plt.figure(figsize=(10, 6))
    plt.plot(df['proba'], df['event'], label='Fraude Acumulado (Event)', color='red', lw=2)
    plt.plot(df['proba'], df['non_event'], label='Legítimo Acumulado (Non-event)', color='blue', lw=2)

    # Línea del KS
    plt.axvline(ks_threshold, color='black', linestyle='--', alpha=0.7)
    plt.text(ks_threshold, 0.5, f'  KS: {ks_stat:.2f}\n  Threshold: {ks_threshold:.4f}',
             fontsize=12, fontweight='bold')

    plt.title(f'Curva de Separación KS (Gini: {gini:.2f})', fontsize=14)
    plt.xlabel('Umbral de Probabilidad (Score)', fontsize=12)
    plt.ylabel('Distribución Acumulada', fontsize=12)
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    plt.gca().invert_xaxis() # Invertimos para ver de mayor a menor probabilidad
    plt.show()

    return ks_stat, ks_threshold

# Ejecutar con tus predicciones
# Usamos el Modelo B (el que tiene scale_pos_weight)
y_probs_b = model_b.predict_proba(X_test)[:, 1]
ks_val, cut_off = plot_ks_gini(y_test, y_probs_b)

In [ ]:
import shap

# Crear el explicador
explainer = shap.TreeExplainer(model_b)
shap_values = explainer.shap_values(X_test)

# Resumen de variables
plt.title("Impacto SHAP: ¿Qué variables empujan al Fraude?")
shap.summary_plot(shap_values, X_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 1. Calcular la matriz
cm = confusion_matrix(y_test, y_pred_ks)

# 2. Configurar el gráfico
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Legítimo (0)', 'Fraude (1)'],
            yticklabels=['Legítimo (0)', 'Fraude (1)'])

plt.title(f'Matriz de Confusión (Umbral KS: {threshold})', fontsize=14)
plt.ylabel('Realidad', fontsize=12)
plt.xlabel('Predicción del Modelo', fontsize=12)

# Añadir etiquetas explicativas
plt.text(0.5, 0.2, 'Verdaderos Negativos\n(Clientes Felices)', ha='center', va='center', color='white', fontweight='bold')
plt.text(1.5, 0.2, 'Falsos Positivos\n(Bloqueos Erróneos)', ha='center', va='center', color='black', fontweight='bold')
plt.text(0.5, 1.2, 'Falsos Negativos\n(Fraude Escapado)', ha='center', va='center', color='black', fontweight='bold')
plt.text(1.5, 1.2, 'Verdaderos Positivos\n(Fraude Atrapado)', ha='center', va='center', color='black', fontweight='bold')

plt.show()